In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
#!pip install kaggle

# Завантаження kaggle.json файлу з API ключем
import os
import json
import shutil
import kaggle

kaggle_json_path = os.path.expanduser('~/.kaggle/kaggle.json')
# kaggle_json_path = 'kaggle.json'
target_path = os.path.expanduser('~/.kaggle/kaggle.json')
# Ensure the .kaggle directory exists
# os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)
os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd

# Завантажуємо тренувальні дані
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')

# Завантажуємо тестові дані
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')

# Перевіримо, як виглядають перші рядки даних
train_data.head()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/titanic/train.csv'

In [3]:
# Дивимося, чи є відсутні значення
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
# Заповнення пропущених значень
# Створюємо нову колонку, яка вказує на те, чи було пропущене значення
train_data['Age_was_missing'] = train_data['Age'].isnull().astype(int)

# Замість заповнення середнім ставимо умовне значення
train_data['Age'] = train_data['Age'].fillna(-1)
train_data['Embarked'] = train_data['Embarked'].fillna(train_data['Embarked'].mode()[0])

# Перевірка, що колонка 'Sex' містить правильні значення
print(train_data['Sex'].unique())
# Перетворюємо колонку 'Sex' на числові значення. чомусь змінює інші значення і таблиці?
#train_data['Sex'] = train_data['Sex'].map({'male': 0, 'female': 1})
# Перетворюємо колонку 'Sex' на числові значення
train_data['Sex'] = train_data['Sex'].replace({'male': 0, 'female': 1})

# Перевірка результату
train_data.isnull().sum()

['male' 'female']


/tmp/ipykernel_17/79739080.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data['Sex'] = train_data['Sex'].replace({'male': 0, 'female': 1})


PassengerId          0
Survived             0
Pclass               0
Name                 0
Sex                  0
Age                  0
SibSp                0
Parch                0
Ticket               0
Fare                 0
Cabin              687
Embarked             0
Age_was_missing      0
dtype: int64

In [5]:
train_data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_was_missing
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S,0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S,0


In [6]:
# Перевірка розподілу класу 'Survived'
print(train_data['Survived'].value_counts(normalize=True))

Survived
0    0.616162
1    0.383838
Name: proportion, dtype: float64


In [7]:
X = train_data[['Pclass', 'Sex', 'Age', 'Fare']]
y = train_data['Survived']

In [8]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

In [9]:
from sklearn.model_selection import train_test_split

# Розділяємо дані на тренувальні і тестові
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Навчаємо модель
model.fit(X_train, y_train)

# Оцінюємо точність моделі на валідаційному наборі
accuracy = model.score(X_val, y_val)
print(f'Training Accuracy: {accuracy}')

Training Accuracy: 0.776536312849162


In [10]:
train_data.isnull().sum()

PassengerId          0
Survived             0
Pclass               0
Name                 0
Sex                  0
Age                  0
SibSp                0
Parch                0
Ticket               0
Fare                 0
Cabin              687
Embarked             0
Age_was_missing      0
dtype: int64

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Передбачаємо значення на валідаційних даних
y_pred = model.predict(X_val)

# Оцінка моделі після тренування
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')


Accuracy: 0.776536312849162
Precision: 0.7428571428571429
Recall: 0.7027027027027027
F1-Score: 0.7222222222222223


In [12]:
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [13]:
# Заповнення пропущених значень для test_data

# Створюємо нову колонку, яка вказує на пропущені значення в 'Age'
test_data['Age_was_missing'] = test_data['Age'].isnull().astype(int)

# Заповнюємо пропущені значення в 'Age' умовним значенням -1
test_data['Age'] = test_data['Age'].fillna(-1)
# Перевіряємо унікальні значення в 'Sex' і перетворюємо на числові значення
test_data['Sex'] = test_data['Sex'].replace({'male': 0, 'female': 1})

# Заповнюємо пропущене значення в колонці 'Fare'
test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].mean())

# Перевіряємо, чи є ще пропущені значення
test_data.isnull().sum()


/tmp/ipykernel_17/4279207399.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data['Sex'] = test_data['Sex'].replace({'male': 0, 'female': 1})


PassengerId          0
Pclass               0
Name                 0
Sex                  0
Age                  0
SibSp                0
Parch                0
Ticket               0
Fare                 0
Cabin              327
Embarked             0
Age_was_missing      0
dtype: int64

In [14]:
predictions = model.predict(test_data[['Pclass', 'Sex', 'Age', 'Fare']])

# Збереження результатів для завантаження на Kaggle
output = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Submission file created!")


Submission file created!
